In [32]:
import pandas as pd
import numpy as np

In [1]:
def euclidean(ai, bi):
    return ((ai - bi))**2

In [2]:
def calcDistance(query, instance, weighted):
    distance = 0
    weight = 1
    for i in range(len(instance)-1):
        distance = distance + euclidean(query[i],instance[i])
    distance = (distance)**0.5
    if (weighted == 1):
        weight = 1/(distance**2)
    return ((distance, weight))

In [3]:
def vdm(ai, bi, attributeCount, pos, classList):
    nia = 0
    nib = 0
    vdmResult = 0
    for i in range(len(attributeCount)):
        if (attributeCount[i][0] == ai and attributeCount[i][2] == pos):
            nia = nia + attributeCount[i][3]
        if (attributeCount[i][0] == bi and attributeCount[i][2] == pos):
            nib = nib + attributeCount[i][3]
                   
    for j in range(len(classList)):
        niac = 0
        nibc = 0
        for i in range(len(attributeCount)):
            if (attributeCount[i][0] == ai and attributeCount[i][1] == classList[j] and attributeCount[i][2] == pos):
                niac = niac + attributeCount[i][3]
            if (attributeCount[i][0] == ai and attributeCount[i][1] == classList[j] and attributeCount[i][2] == pos):
                nibc = nibc + attributeCount[i][3]
        
        if (nia == 0):
            v1 = 0
        else:
            v1 = niac/nia
        
        if (nib == 0):
            v2 = 0
        else:
            v2 = nibc/nib
        
        calc = abs(v1-v2)
        vdmResult = vdmResult + calc
        
    return vdmResult

In [4]:
def calcVDM(query, instance, weighted, attributeCount, classList):
    distance = 0
    weight = 1
    for i in range(len(query)):
        distance = distance + vdm(query[i], instance[i], attributeCount, i, classList)
    
    distance = (distance)**0.5
    
    if (weighted == 1):
        weight = 1/(distance**2)
    
    return ((distance, weight))

In [5]:
def calcHVDM(query, instance, weighted, attributeCount, classList, typeList):
    distance = 0
    weight = 1
    for i in range(len(query)):
        if (typeList[i] == 'num'):
            distance = distance + ((euclidean(query[i], instance[i]))**2)
        elif (typeList[i] == 'cat'):
            distance = distance + ((vdm(query[i], instance[i], attributeCount, i, classList))**2)
    
    distance = (distance)**0.5
    
    if (weighted == 1):
        weight = 1/(distance**2)
    
    return ((distance, weight))

In [6]:
def knn (query, n, data, weighted, dataType, typeList):
    # k = testInstance
    # 
    #dataType 0 = numérico, 1 = categorico, 2 = heterogeneo
    distances = []
    classes = []
    
    attributeCount = []
    
    classColumn = [row[-1] for row in data]
    classList = []
    
    for i in range(len(classColumn)):
        if (classColumn[i] not in classList):
            classList.append(classColumn[i])
    
    for i in range(len(data[0])-1):
        for j in range(len(data)):
            flag = 0
            for k in range(len(attributeCount)):
                if (data[j][i]==attributeCount[k][0] and data[j][-1]==attributeCount[k][1] and i == attributeCount[k][2]):
                    attributeCount[k][3] = attributeCount[k][3] + 1
                    flag = 1
                
            if(flag == 0):
                attributeCount.append([data[j][i], data[j][-1], i, 1])
            
            #if (data[j][i] not in attr):
             #   if (len(attr)==0):
              #      attributeCount.append([data[j][i], data[j][-1], 1])
               # else:
                #    for k in range(len(attributeCount)):
                 #       if (data[j][i]==attributeCount[k][0] and data[j][-1]==attributeCount[k][1]):
                  #          attributeCount[k][2] = attributeCount[k][2] + 1
                #attr.append([data[j][i], data[j][-1]])

    #euclidean
    if (dataType == 0):
        for i in range(len(data)):
            distance = calcDistance(query, data[i], weighted)
            distances.append((data[i], distance[0], distance[1]))
        distances.sort(key=lambda x: x[0], reverse=True)
        distances.sort(key=lambda x: x[1], reverse=False)
    #vdm
    elif (dataType == 1):
        for i in range(len(data)):
            distance = calcVDM(query, data[i], weighted, attributeCount, classList)
            distances.append((data[i], distance[0], distance[1]))
        distances.sort(key=lambda x: x[0], reverse=True)
        distances.sort(key=lambda x: x[1], reverse=False)
    
    #hvdm
    elif (dataType == 2):
        for i in range(len(data)):
            distance = calcHVDM(query, data[i], weighted, attributeCount, classList, typeList)
            distances.append((data[i], distance[0], distance[1]))
        distances.sort(key=lambda x: x[0], reverse=True)
        distances.sort(key=lambda x: x[1], reverse=False)
    
    classList = []
    for i in range(n):
        myClass = distances[i][0][-1]
        myWeight = distances[i][2]
        if (myClass not in classList):
            classes.append((myClass, myWeight))
            classList.append(myClass)
        else:
            for j in range(len(classes)):
                if (myClass == classes[j][0]):
                    classes[j] = list(classes[j])
                    classes[j][1] = classes[j][1] + myWeight
                    break
    
    classes.sort(key=lambda x: x[1], reverse=True)
    for i in range(len(distances)):
        print (distances[i])
    return classes

In [59]:
import random

In [58]:
def kFolds (dataset, k, shuffle, mySeed):
    
    finalDataset = dataset.copy()
    if (shuffle==1):
        random.seed(mySeed)
        finalDataset = dataset.sample(frac=1, random_state=mySeed)
        
    splittedDataset = np.array_split(finalDataset, k)
    
    return splittedDataset

In [88]:
def crossValidation(dataset, kFold, shuffle, mySeed, k, weighted, dataType):
    
    typeList = []
    foldHits = []
    datasetFolds = kFolds(dataset, kFold, shuffle, mySeed)
    for i in range(kFold):
        testset = datasetFolds[i]
        frames = []
        for j in range(kFold):
            if (not j==i):
                frames.append(datasetFolds[j])
        trainset = pd.concat(frames)
        hits = 0
        for m in range(len(testset)):
            hit = 0
            #for z in range(len())
            print(testset[m])
            neighbors = knn(testset[m], k, trainset, weighted, dataType, typeList)
            if (neighbors[0][0] == testset[m][-1]):
                hit = 1
            hits = hits + hit
        foldHits.append(hits)
    return foldHits    

In [89]:
abb = crossValidation(cm1, 5, 1, 1000, 5, 0, 0)

KeyError: 0

In [60]:
cm1 = pd.read_csv("..//datasets/q1/cm1.csv")
kc2 = pd.read_csv("..//datasets/q1/kc2.csv")
car = pd.read_csv("..//datasets/q2/car.csv")
hayesRoth = pd.read_csv("..//datasets/q2/hayes-roth.csv")
abalone = pd.read_csv("..//datasets/q3/abalone.csv")
crx = pd.read_csv("..//datasets/q3/crx.csv")

In [68]:
cm1Folds = kFolds(cm1, 5, 1, 1000)
for i in range(5):
    testset = cm1Folds[i]
    frames = []
    for j in range(5):
        if (not j==i):
            frames.append(cm1Folds[j])
    trainset = pd.concat(frames)
    

100
100
100
99
99


In [38]:
random.seed(1000)
random.randint(1, 10)

7

In [12]:
trainset = [[0, 0, 1, 'a'], [0, 1, 0, 'b'], [0,0,0, 'a'],[1,1,1,'b'],[1,1,0,'a']]
testInstance = [1, 0, 1]
k = 1
neighbors = knn(testInstance, 3, trainset, 0, 0, ['num','num','num','num','num'])
print(neighbors)
neighbors[0][0]

([1, 1, 1, 'b'], 1.0, 1)
([0, 0, 1, 'a'], 1.0, 1)
([1, 1, 0, 'a'], 1.4142135623730951, 1)
([0, 0, 0, 'a'], 1.4142135623730951, 1)
([0, 1, 0, 'b'], 1.7320508075688772, 1)
[['a', 2], ('b', 1)]


'a'

In [ ]:
## read csv -> k-fold -> joga no metodo -> calcula metricas

In [33]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.ticker import FormatStrFormatter
import seaborn as sns

In [42]:
cm1a = cm1.sample(frac=1, random_state=1000)

In [47]:
splitCm1 = np.array_split(cm1a, 5)

In [53]:
splitCm1[4]

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,A13,A14,A15,A16,A17,A18,A19,A20,A21,A22
13,23.0,7.0,5.0,1.0,105.0,474.97,0.04,27.22,17.45,12929.85,...,10,17,23,0,14.0,9.0,70.0,35.0,13.0,False
201,30.0,4.0,1.0,2.0,193.0,1149.16,0.04,25.34,45.35,29117.23,...,10,22,21,0,25.0,37.0,118.0,75.0,7.0,False
386,9.0,1.0,1.0,1.0,21.0,84.00,0.19,5.14,16.32,432.00,...,0,3,5,0,9.0,7.0,13.0,8.0,1.0,False
176,8.0,1.0,1.0,1.0,24.0,88.81,0.14,7.20,12.33,639.44,...,0,0,0,0,8.0,5.0,15.0,9.0,1.0,False
200,3.0,1.0,1.0,1.0,7.0,19.65,0.40,2.50,7.86,49.13,...,0,0,0,0,5.0,2.0,5.0,2.0,1.0,False
104,34.0,6.0,1.0,6.0,146.0,771.67,0.08,12.00,64.31,9260.02,...,4,0,10,0,12.0,27.0,92.0,54.0,11.0,False
193,3.0,1.0,1.0,1.0,7.0,18.09,0.40,2.50,7.24,45.24,...,0,0,0,0,5.0,1.0,6.0,1.0,1.0,False
20,7.0,2.0,1.0,1.0,27.0,105.49,0.08,12.00,8.79,1265.83,...,0,0,1,0,10.0,5.0,15.0,12.0,3.0,False
56,25.0,4.0,1.0,1.0,95.0,475.00,0.07,15.00,31.67,7125.00,...,0,8,7,0,16.0,16.0,65.0,30.0,7.0,False
428,9.0,1.0,1.0,1.0,81.0,366.41,0.05,20.22,18.12,7409.59,...,0,2,3,0,14.0,9.0,55.0,26.0,1.0,False
